In [51]:
import requests
API_KEY = '1b28274d-1b90-43c3-ad36-dd730905b034'
headers = {'apikey': API_KEY}
# Fetch data from APIs
test_info = requests.get('https://api.learnbasics.fun/training/test/info/',headers=headers).json()
student_details = requests.get('https://api.learnbasics.fun/training/students/', headers=headers).json()
test_data = requests.get('https://api.learnbasics.fun/training/test/data/', headers=headers).json()
concept_data = requests.get('https://api.learnbasics.fun/training/test/concepts/',headers=headers).json()


import pandas as pd
df_students = pd.DataFrame(student_details)
df_test_data = pd.DataFrame(test_data)

#df_students.info()
#df_students.head()
#df_test_data.info()
#df_test_data[:30]

In [59]:
test_info['']

dict_keys(['test_name', 'test_url', 'section', 'class', 'subject', 'school_name', 'school_acronym', 'chapter_name', 'start_time', 'end_time', 'concept_coveblack'])

In [40]:
def convert(x):
    days = int(x)
    return f'{days} Day Ago'
def convert_to_fraction(x):
    total_marks_test = len(test_df['question_id'].unique())
    if x == 'Absent':
        return x
    else:
        return f"{int(x)}/{total_marks_test}"

test_df=df_test_data
test_summary = test_df.groupby('learner_id').agg(
    total_score=('mark', 'sum'),
    total_time_taken=('time_taken', 'sum'),
    total_attempts=('attempt','mean')
).reset_index()

#final_summary = pd.merge(test_summary, total_marks_test, on='learner_id')
test_summary['total_time_taken'] = pd.to_datetime(test_summary['total_time_taken'], unit='s').dt.strftime('%M:%S')

# Merge with the student table
final_output = pd.merge(df_students, test_summary, on='learner_id', how='left')

# Convert last_login to the desired format
final_output['last_login'] = final_output['last_login'].apply(convert)

# Fill NaN values in total_score and total_time_taken for absent students
final_output['total_score'] = final_output['total_score'].fillna('Absent')
final_output['total_time_taken'] = final_output['total_time_taken'].fillna('')
final_output['total_score']=final_output['total_score'].apply(convert_to_fraction)



class_performance=final_output
class_performance = class_performance[['student_name', 'last_login', 'total_score', 'total_time_taken']]




class_performance = class_performance.reset_index(drop=False)
#class_performance.set_index(('S_no',axis=1)
class_performance['index']=class_performance['index']+1
class_performance.columns = ['S_no','Name', 'Last Login', 'Score', 'Time Taken (mm:ss)']

class_performance_data = [class_performance.columns.tolist()] + class_performance.values.tolist()

final_output

,student_name,learner_id,last_login,total_score,total_time_taken,total_attempts
0,Athya,8307367,2 Day Ago,Absent,,NaN
1,Bharathh,8307368,2 Day Ago,Absent,,NaN
2,Chinthanika,8307369,2 Day Ago,1/5,02:01,1.0
3,Divya,8307370,3 Day Ago,Absent,,NaN
4,Glaxan,8307371,1 Day Ago,Absent,,NaN
5,Harshitha,8307372,1 Day Ago,4/5,06:01,1.0
6,Hemavarsha,8307373,3 Day Ago,5/5,03:20,1.0
7,Kanishka,8307375,1 Day Ago,3/5,05:17,1.0
8,Kritik,8307376,2 Day Ago,2/5,00:22,1.0
9,Manirathnam,8307377,3 Day Ago,Absent,,NaN


In [84]:
ques_summary = df_test_data.groupby('question_id').agg(
    total_score=('mark', 'sum'),
    total_attempts=('attempt','sum')
).reset_index()
ques_summary['percentage']=(ques_summary['total_score']/ques_summary['total_attempts']*100).astype('int64')
ques_summary=ques_summary.drop(['total_score','total_attempts'],axis=1)
ques_summary.columns=['Question Number', 'Performance']
ques_summary['Question Number'] = [f'Q{i+1}' for i in range(ques_summary.shape[0])]
ques_summary=ques_summary.T
ques_summary.columns = ques_summary.iloc[0]
#ques_summary.columns = ["Q1", "Q2", "Q3", "Q4", "Q5"]
#ques_summary = ques_summary[1:]
ques_summary.reset_index(inplace=True)

class_performance_data = [ques_summary.columns.tolist()] + ques_summary.values.tolist()
class_performance_data[1:]

[['Question Number', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5'],
 ['Performance', 92, 76, 53, 46, 69]]